# Crossing the Streams – Joins

## Intro

We are going to use an example to demonstrate the differences in the joins. It is based on the online advertising domain. There is a Kafka topic that contains view events of particular ads and another one that contains click events based on those ads. Views and clicks share an ID that serves as the key in both topics.

**Apache Spark joins**:

- Stream-static joins: INNER and LEFT-OUTER stream-static joins are not statuful; OUTER and RIGHT-OUTER are not supported

- Stream-stream joins:
    - Inner: Supported, optionally specify watermark on both sides + time constraints for state cleanup
    - Left Outer: Conditionally supported, must specify watermark on right + time constraints for correct results, optionally specify watermark on left for all state cleanup
    - Right Outer: Conditionally supported, must specify watermark on left + time constraints for correct results, optionally specify watermark on right for all state cleanup
    - Full Outer: Not supported

In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
import io
from pyspark.sql.functions import *
import time
import json
import struct
import requests 
from IPython.display import clear_output

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1,org.apache.spark:spark-streaming-kafka-0-10_2.11:2.4.5,org.apache.kafka:kafka-clients:2.6.0 pyspark-shell'
                                    
spark = (SparkSession.builder 
    .master("local[*]")
    .appName("test")
    .getOrCreate()
        )

spark

set up the environment variables

In [2]:
advertisement_topic = 'ParticularAdEvent'
click_topic = 'ClickEvent'
servers = "kafka:9092"

### Let's create the streaming Data Frames using the data in the kafka topics

In [3]:
raw_streaming_ad_df = (spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", servers)
  .option("startingOffsets", "earliest")
  .option("subscribe", advertisement_topic)
  .load())

In [5]:
raw_streaming_ad_df.isStreaming

True

In [6]:
raw_streaming_ad_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [7]:
from pyspark.sql.types import *

ad_schema = StructType([
    StructField("ad", StringType(), True),
    StructField("ts", TimestampType(), True)])

In [8]:
ad_sdf=(raw_streaming_ad_df
                      .select(from_json(col("value").cast("string"), ad_schema).alias("value"))
                      .select("value.*"))

In [9]:
ad_sdf.printSchema()

root
 |-- ad: string (nullable = true)
 |-- ts: timestamp (nullable = true)



In [10]:
click_schema = StructType([
    StructField("choose_ad", StringType(), True),
    StructField("ts", TimestampType(), True)])

raw_streaming_click_df = (spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", servers)
  .option("startingOffsets", "earliest")
  .option("subscribe", click_topic)
  .load())

click_sdf = (raw_streaming_click_df
                      .select(from_json(col("value").cast("string"), click_schema).alias("value"))
                      .select("value.*"))

### Before we start to work with queries, let's inspect the content of our advertisement stream

In [11]:
basic_query = (ad_sdf
    .writeStream
    .format("memory") # this is for debug purpose only! DO NOT USE IN PRODUCTION
    .queryName("sinkTable_basic")
    .start())

In [14]:
spark.sql("SELECT * FROM sinkTable_basic ORDER BY TS DESC").show(5)

+---+-------------------+
| ad|                 ts|
+---+-------------------+
|  G|2022-04-26 10:41:18|
|  F|2022-04-26 10:41:12|
|  D|2022-04-26 10:41:06|
|  C|2022-04-26 10:41:00|
|  B|2022-04-26 10:40:54|
+---+-------------------+
only showing top 5 rows



In [15]:
basic_query.stop()

### Q1 - Inner Stream-Stream Join

In [16]:
last_minute_ad_events = (ad_sdf
                .withWatermark("ts", "1 minute")
                .withColumnRenamed("ad","advertisement")
                .withColumnRenamed("ts","tsAdvertisement")
               )

last_minute_click_events = (click_sdf
                .withWatermark("ts", "1 minute")
                .withColumnRenamed("ts","tsClick")
               )

We use the interval equal to 10 seconds

In [17]:
join_sdf = (last_minute_ad_events.join(
  last_minute_click_events, expr("""
    (advertisement == choose_ad) AND
    (tsClick > tsAdvertisement ) AND
    (tsClick < tsAdvertisement + interval 10 seconds )
    """
    )))

In [18]:
q1 = (join_sdf
            .writeStream
            .format("memory")
            .queryName("sinkTable")
            .start())

In [20]:
spark.sql("SELECT * FROM sinkTable ORDER BY tsClick DESC").show(20,False) # note, I change ts in tsClick

+-------------+-------------------+---------+-------------------+
|advertisement|tsAdvertisement    |choose_ad|tsClick            |
+-------------+-------------------+---------+-------------------+
|G            |2022-04-26 10:40:42|G        |2022-04-26 10:40:48|
|D            |2022-04-26 10:39:54|D        |2022-04-26 10:40:02|
|B            |2022-04-26 10:39:06|B        |2022-04-26 10:39:07|
|G            |2022-04-26 10:38:17|G        |2022-04-26 10:38:22|
|C            |2022-04-26 10:36:47|C        |2022-04-26 10:36:51|
|A            |2022-04-26 10:35:59|A        |2022-04-26 10:36:08|
+-------------+-------------------+---------+-------------------+



In [21]:
q1.stop()

### Q2 - Left Stream-Stream Join

In [22]:
last_minute_ad_events_left = (ad_sdf
                .withWatermark("ts", "3 seconds")
                .withColumnRenamed("ad","advertisement")
                .withColumnRenamed("ts","tsAdvertisement")
               )

last_minute_click_events_left = (click_sdf
                .withWatermark("ts", "3 seconds")
                .withColumnRenamed("ts","tsClick")
               )

In [23]:
join_sdf_second = (last_minute_ad_events_left.join(
  last_minute_click_events_left, expr("""
    (advertisement == choose_ad) AND
    (tsClick > tsAdvertisement ) AND
    (tsClick < tsAdvertisement + interval 3 seconds )
    """
    ), "leftOuter"))

In [24]:
q2 = (join_sdf_second
            .writeStream
            .format("memory")
            .queryName("sinkTable")
            .start())

In [26]:
spark.sql("SELECT * FROM sinkTable ORDER BY tsClick DESC").show(10) # note, I change ts in tsClick

+-------------+-------------------+---------+-------------------+
|advertisement|    tsAdvertisement|choose_ad|            tsClick|
+-------------+-------------------+---------+-------------------+
|            B|2022-04-26 10:39:06|        B|2022-04-26 10:39:07|
|            F|2022-04-26 10:35:47|     null|               null|
|            F|2022-04-26 10:41:48|     null|               null|
|            F|2022-04-26 10:36:23|     null|               null|
|            F|2022-04-26 10:39:24|     null|               null|
|            F|2022-04-26 10:41:12|     null|               null|
|            F|2022-04-26 10:40:36|     null|               null|
|            F|2022-04-26 10:40:00|     null|               null|
|            F|2022-04-26 10:38:48|     null|               null|
|            F|2022-04-26 10:38:11|     null|               null|
+-------------+-------------------+---------+-------------------+
only showing top 10 rows



In [27]:
q2.stop()

### Q3 - Outer Stream-Stream Join

Full outer join is not supported while doing stream/stream joins in Apache Spark Structured Streaming. To show that it is true, we should execute the following cells

In [28]:
join_sdf = (last_minute_ad_events.join(
  last_minute_click_events, expr("""
    (advertisement == choose_ad) AND
    (tsClick > tsAdvertisement ) AND
    (tsClick < tsAdvertisement + interval 10 seconds )
    """
    ), "fullOuter"))

In [29]:
q3 = (join_sdf
            .writeStream
            .format("memory")
            .queryName("sinkTable")
            .start())

AnalysisException: Full outer joins with streaming DataFrames/Datasets are not supported;;
Join FullOuter, (((advertisement#115 = choose_ad#50) AND (tsClick#121-T60000ms > tsAdvertisement#118-T60000ms)) AND (tsClick#121-T60000ms < cast(tsAdvertisement#118-T60000ms + 10 seconds as timestamp)))
:- Project [advertisement#115, ts#24-T60000ms AS tsAdvertisement#118-T60000ms]
:  +- Project [ad#23 AS advertisement#115, ts#24-T60000ms]
:     +- EventTimeWatermark ts#24: timestamp, 1 minutes
:        +- Project [value#21.ad AS ad#23, value#21.ts AS ts#24]
:           +- Project [from_json(StructField(ad,StringType,true), StructField(ts,TimestampType,true), cast(value#8 as string), Some(Etc/UTC)) AS value#21]
:              +- StreamingRelationV2 org.apache.spark.sql.kafka010.KafkaSourceProvider@68fb8728, kafka, org.apache.spark.sql.kafka010.KafkaSourceProvider$KafkaTable@6c87f986, org.apache.spark.sql.util.CaseInsensitiveStringMap@fdaffb46, [key#7, value#8, topic#9, partition#10, offset#11L, ti

### Preparation to create table

The feature of joining static tables is supported in Apache SSS but it is **not the same** as **joining Ktables**. 

Before we start with joining KTable & KTable, we should remember that Apache Spark doesn't have a totally equivalent structure. To create a similar structure for Spark to use, we should respect the following characteristics of KTable:

- KTable contains the latest message only for the corresponding key
- KTable is the always updating table, we can view it as the changelog.

To create a similar structure, we should:

1) Make a spark static table 

2) Updating it by batches

3) Drop the key duplicate

But with creating a spark static table, we will read only the earliest data. We cannot put in startingOffsets - "latest" because this startingOffset is not available for reading batches. 

So, our implementation will be a little bit tricky, since we **keep** the previous values while joining (because as told earlier we can't just read the latest value)

In [30]:
def create_static_tables():
    raw_static_ad_df = (spark
        .read
        .format("kafka")
        .option("kafka.bootstrap.servers", servers)
        .option("startingOffsets", "earliest")
        .option("subscribe", advertisement_topic)
        .load())

    ad_static_df=(raw_static_ad_df
                      .select(from_json(col("value").cast("string"), ad_schema).alias("value"))
                      .select("value.*"))

    raw_static_click_df = (spark
          .read
          .format("kafka")
          .option("kafka.bootstrap.servers", servers)
          .option("startingOffsets", "earliest")
          .option("subscribe", click_topic)
          .load())

    click_static_df = (raw_static_click_df
                      .select(from_json(col("value").cast("string"), click_schema).alias("value"))
                      .select("value.*"))
    
    return ad_static_df, click_static_df

### Q4 - Table-Table Join

**IMPORTANT**:
You have to wait here a little bit longer than in streaming case

In [31]:
while True:
    ad_static_df, click_static_df = create_static_tables()
    
    last_minute_ad_events_static = (ad_static_df
                .withColumnRenamed("ad","advertisement")
                .withColumnRenamed("ts","tsAdvertisement")
               )

    last_minute_click_events_static = (click_static_df
                .withColumnRenamed("ts","tsClick")
               )
    
    join_sdf_fourth = (last_minute_ad_events_static.join(
        last_minute_click_events_static, expr("""
            (advertisement == choose_ad) AND
            (tsClick > tsAdvertisement ) AND
            (tsClick < tsAdvertisement + interval 10 seconds )
            """
        ), "inner"))
    
    join_sdf_fourth.sort(join_sdf_fourth.tsClick.desc()).drop_duplicates(["advertisement"]).show(10, False)
    clear_output(wait=True)

KeyboardInterrupt: 

### Q5 - Left Table-Table Join


**IMPORTANT**:
You have to wait here a little bit longer than in streaming case

In [32]:
while True:
    ad_static_df, click_static_df = create_static_tables()
    
    last_minute_ad_events_static = (ad_static_df
                .withColumnRenamed("ad","advertisement")
                .withColumnRenamed("ts","tsAdvertisement")
               )

    last_minute_click_events_static = (click_static_df
                .withColumnRenamed("ts","tsClick")
               )
    
    join_sdf_five = (last_minute_ad_events_static.join(
      last_minute_click_events_static, expr("""
        (advertisement == choose_ad) AND
        (tsClick > tsAdvertisement ) AND
        (tsClick < tsAdvertisement + interval 5 seconds )
        """
        ), "leftOuter"))    
    
    join_sdf_five.sort(join_sdf_five.tsClick.desc()).drop_duplicates(["advertisement"]).show(10, False)
    clear_output(wait=True)

KeyboardInterrupt: 

### Q6 - Outer Table-Table Join

**IMPORTANT**:
You have to wait here a little bit longer than in streaming case

In [33]:
while True:
    ad_static_df, click_static_df = create_static_tables()
    
    last_minute_ad_events_static = (ad_static_df
                .withColumnRenamed("ad","advertisement")
                .withColumnRenamed("ts","tsAdvertisement")
               )

    last_minute_click_events_static = (click_static_df
                .withColumnRenamed("ts","tsClick")
               )
    
    join_sdf_six = (last_minute_ad_events_static.join(
      last_minute_click_events_static, expr("""
        (advertisement == choose_ad) AND
        (tsClick > tsAdvertisement ) AND
        (tsClick < tsAdvertisement + interval 10 seconds )
        """
        ), "fullOuter"))    
    
    join_sdf_six.sort(join_sdf_six.tsClick.desc()).drop_duplicates(["advertisement"]).show(10, False)
    clear_output(wait=True)

KeyboardInterrupt: 

### Q7 - Inner Stream-Table Join

The assumption here is the following:
- The data that is going from advertisement is static, meaning it is passive, the user is active, so it is modeled as the stream

In [34]:
raw_static_ad_df = (spark
        .read
        .format("kafka")
        .option("kafka.bootstrap.servers", servers)
        .option("startingOffsets", "earliest")
        .option("subscribe", advertisement_topic)
        .load())

ad_static_df=(raw_static_ad_df
                      .select(from_json(col("value").cast("string"), ad_schema).alias("value"))
                      .select("value.*"))

In [35]:
last_minute_ad_events = (ad_static_df
                .withColumnRenamed("ad","advertisement")
                .withColumnRenamed("ts","tsAdvertisement")
               )

last_minute_click_events = (click_sdf
                .withWatermark("ts", "1 minute")
                .withColumnRenamed("ts","tsClick")
               )

In [36]:
join_sdf_seven = (last_minute_click_events.join(
  last_minute_ad_events, expr("""
    (advertisement == choose_ad) AND
    (tsClick > tsAdvertisement ) AND
    (tsClick < tsAdvertisement + interval 10 seconds )
    """
    )))

In [37]:
q7 = (join_sdf_seven
            .writeStream
            .format("memory")
            .queryName("sinkTable")
            .start())

In [42]:
spark.sql("SELECT * FROM sinkTable ORDER BY tsClick DESC").show(20,False) # note, I change ts in tsClick

+---------+-------------------+-------------+-------------------+
|choose_ad|tsClick            |advertisement|tsAdvertisement    |
+---------+-------------------+-------------+-------------------+
|D        |2022-04-26 10:48:22|D            |2022-04-26 10:48:19|
|C        |2022-04-26 10:47:45|C            |2022-04-26 10:47:36|
|D        |2022-04-26 10:45:23|D            |2022-04-26 10:45:18|
|B        |2022-04-26 10:41:38|B            |2022-04-26 10:41:30|
|G        |2022-04-26 10:40:48|G            |2022-04-26 10:40:42|
|D        |2022-04-26 10:40:02|D            |2022-04-26 10:39:54|
|B        |2022-04-26 10:39:07|B            |2022-04-26 10:39:06|
|G        |2022-04-26 10:38:22|G            |2022-04-26 10:38:17|
|C        |2022-04-26 10:36:51|C            |2022-04-26 10:36:47|
|A        |2022-04-26 10:36:08|A            |2022-04-26 10:35:59|
+---------+-------------------+-------------+-------------------+



In [43]:
q7.stop()

### Q8 - Left Stream-Table Join

In [44]:
last_minute_ad_events = (ad_static_df
                .withColumnRenamed("ad","advertisement")
                .withColumnRenamed("ts","tsAdvertisement")
               )

last_minute_click_events = (click_sdf
                .withWatermark("ts", "1 minute")
                .withColumnRenamed("ts","tsClick")
               )

In [45]:
join_sdf_eight = (last_minute_click_events.join(
  last_minute_ad_events, expr("""
    (advertisement == choose_ad) AND
    (tsClick > tsAdvertisement ) AND
    (tsClick < tsAdvertisement + interval 10 seconds )
    """
    ), "leftOuter"))

In [46]:
q8 = (join_sdf_eight
            .writeStream
            .format("memory")
            .queryName("sinkTable")
            .start())

In [47]:
spark.sql("SELECT * FROM sinkTable ORDER BY tsClick DESC").show(20,False) # note, I change ts in tsClick

+---------+-------------------+-------------+-------------------+
|choose_ad|tsClick            |advertisement|tsAdvertisement    |
+---------+-------------------+-------------+-------------------+
|A        |2022-04-26 10:49:32|null         |null               |
|G        |2022-04-26 10:48:55|null         |null               |
|F        |2022-04-26 10:48:35|null         |null               |
|D        |2022-04-26 10:48:22|D            |2022-04-26 10:48:19|
|C        |2022-04-26 10:47:45|C            |2022-04-26 10:47:36|
|B        |2022-04-26 10:47:16|null         |null               |
|A        |2022-04-26 10:46:44|null         |null               |
|G        |2022-04-26 10:46:28|null         |null               |
|F        |2022-04-26 10:45:57|null         |null               |
|D        |2022-04-26 10:45:23|D            |2022-04-26 10:45:18|
|C        |2022-04-26 10:44:55|null         |null               |
|B        |2022-04-26 10:44:17|null         |null               |
|A        

In [48]:
q8.stop()

### Q9 - Inner Stream-GlobalKTable Join

Natively **GlobalKTable** is **not supported** in Apache SSS.

GlobalKTable should be populated before any processing is done we should have a very big buffer which, of course in real processing is hard to choose because of its size mainly, we need to know in advance its size.

To model it we can use a static dataframe, that has already all the data inside of it, **BUT** this solution is infeasible in the real world. Meaning that we will do again **static-stream** join that is supported in Apache Spark Structured Streaming to emulate GlobalKTable

Again we assume that the user is active when the advertisement are passive

**IMPORTANT**: Notice that in Q9 and Q10 time intervals increased it is done due to simulator of click events that has the delay of user clicking on the advertisement

In [49]:
from pyspark import SparkConf
from pyspark import SparkContext
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

#create the static-df
global_table = sc.parallelize([
    ['A', time.time()],
    ['B', time.time()],
    ['C', time.time()],
    ['D', time.time()],
    ['F', time.time()],
    ['G', time.time()],
]
).toDF(["advertisement","tsAdvertisement"])

In [50]:
global_table = global_table.withColumn("tsAdvertisement", to_timestamp(global_table["tsAdvertisement"]))
global_table.show(4, False)

+-------------+--------------------------+
|advertisement|tsAdvertisement           |
+-------------+--------------------------+
|A            |2022-04-26 10:51:27.604575|
|B            |2022-04-26 10:51:27.604576|
|C            |2022-04-26 10:51:27.604576|
|D            |2022-04-26 10:51:27.604576|
+-------------+--------------------------+
only showing top 4 rows



In [51]:
join_sdf_nine = last_minute_click_events.join(global_table, 
                                                    expr("""
                                                        (advertisement == choose_ad) AND
                                                        (tsClick > tsAdvertisement ) AND
                                                        (tsClick < tsAdvertisement + interval 60 seconds )
                                                        """
                                                    ))

In [52]:
q9 = (join_sdf_nine
            .writeStream
            .format("memory")
            .queryName("sinkTable")
            .start())

In [56]:
spark.sql("SELECT * FROM sinkTable ORDER BY tsClick DESC").show(4,False) # note, I change ts in tsClick

+---------+-------------------+-------------+--------------------------+
|choose_ad|tsClick            |advertisement|tsAdvertisement           |
+---------+-------------------+-------------+--------------------------+
|G        |2022-04-26 10:51:45|G            |2022-04-26 10:51:27.604577|
+---------+-------------------+-------------+--------------------------+



In [57]:
q9.stop()

### Q10 - Left Stream-GlobalKTable Join

Since **GlobalKTable** is not supportd in Apache SSS we can't do this join. But we can emulate it using a static dataframe like in the previous **Q9**

In [79]:
#create the static-df
global_table = sc.parallelize([
    ['A', time.time()],
    ['B', time.time()],
]
).toDF(["advertisement","tsAdvertisement"])

In [80]:
global_table = global_table.withColumn("tsAdvertisement", to_timestamp(global_table["tsAdvertisement"]))
global_table.show(2, False)

+-------------+--------------------------+
|advertisement|tsAdvertisement           |
+-------------+--------------------------+
|A            |2022-04-26 10:53:02.702227|
|B            |2022-04-26 10:53:02.702227|
+-------------+--------------------------+



In [81]:
join_sdf_ten = last_minute_click_events.join(global_table, 
                                                    expr("""
                                                        (advertisement == choose_ad) AND
                                                        (tsClick > tsAdvertisement ) AND
                                                        (tsClick < tsAdvertisement + interval 20 seconds )
                                                        """
                                                    ), "leftOuter")

In [82]:
q10 = (join_sdf_ten
            .writeStream
            .format("memory")
            .queryName("sinkTable")
            .start())

In [86]:
spark.sql("SELECT * FROM sinkTable ORDER BY tsClick DESC").show(8,False) # note, I change ts in tsClick

+---------+-------------------+-------------+---------------+
|choose_ad|tsClick            |advertisement|tsAdvertisement|
+---------+-------------------+-------------+---------------+
|B        |2022-04-26 10:52:33|null         |null           |
|A        |2022-04-26 10:52:16|null         |null           |
|G        |2022-04-26 10:51:45|null         |null           |
|F        |2022-04-26 10:51:10|null         |null           |
|D        |2022-04-26 10:50:53|null         |null           |
|C        |2022-04-26 10:50:19|null         |null           |
|B        |2022-04-26 10:49:42|null         |null           |
|A        |2022-04-26 10:49:32|null         |null           |
+---------+-------------------+-------------+---------------+
only showing top 8 rows



In [78]:
q10.stop()